In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly, sklearn_sm, Stepwise, sklearn_selected, sklearn_selection_path)
from functools import partial
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm

c:\Users\admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [11]:
from sklearn.model_selection import ShuffleSplit, cross_validate
from sklearn.tree import DecisionTreeClassifier as DTC, DecisionTreeRegressor as DCR, plot_tree, export_text
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestRegressor as RFR, GradientBoostingRegressor as GradientBoostingRegressor
from ISLP.bart import BART

In [3]:
carseats = load_data('Carseats')
High = np.where(carseats['Sales'] > 8, 'Yes', 'No')

In [6]:
model = MS(carseats.columns.drop('Sales'), intercept=False)
D = model.fit_transform(carseats)
feature_names = list(D.columns)
X = np.asarray(D)
X, D

(array([[138.,  73.,  11., ...,  17.,   1.,   1.],
        [111.,  48.,  16., ...,  10.,   1.,   1.],
        [113.,  35.,  10., ...,  12.,   1.,   1.],
        ...,
        [162.,  26.,  12., ...,  18.,   1.,   1.],
        [100.,  79.,   7., ...,  12.,   1.,   1.],
        [134.,  37.,   0., ...,  16.,   1.,   1.]]),
      CompPrice  Income  Advertising  Population  Price  ShelveLoc[Good]  \
 0          138      73           11         276    120              0.0   
 1          111      48           16         260     83              1.0   
 2          113      35           10         269     80              0.0   
 3          117     100            4         466     97              0.0   
 4          141      64            3         340    128              0.0   
 ..         ...     ...          ...         ...    ...              ...   
 395        138     108           17         203    128              1.0   
 396        139      23            3          37    120              0.

In [7]:
clf = DTC(criterion='entropy', max_depth=3, random_state=0)
clf.fit(X, High)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

In [8]:
accuracy_score(High, clf.predict(X))

0.79

In [9]:
print(export_text(clf, feature_names=feature_names, show_weights=True))

|--- ShelveLoc[Good] <= 0.50
|   |--- Price <= 92.50
|   |   |--- Income <= 57.00
|   |   |   |--- weights: [7.00, 3.00] class: No
|   |   |--- Income >  57.00
|   |   |   |--- weights: [7.00, 29.00] class: Yes
|   |--- Price >  92.50
|   |   |--- Advertising <= 13.50
|   |   |   |--- weights: [183.00, 41.00] class: No
|   |   |--- Advertising >  13.50
|   |   |   |--- weights: [20.00, 25.00] class: Yes
|--- ShelveLoc[Good] >  0.50
|   |--- Price <= 135.00
|   |   |--- US[Yes] <= 0.50
|   |   |   |--- weights: [6.00, 11.00] class: Yes
|   |   |--- US[Yes] >  0.50
|   |   |   |--- weights: [2.00, 49.00] class: Yes
|   |--- Price >  135.00
|   |   |--- Income <= 46.00
|   |   |   |--- weights: [6.00, 0.00] class: No
|   |   |--- Income >  46.00
|   |   |   |--- weights: [5.00, 6.00] class: Yes



In [12]:
valid = ShuffleSplit(n_splits=1, test_size=200, random_state=0)
results = cross_validate(clf, D, High, cv=5)
results['test_score']

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

array([0.6875, 0.6875, 0.7125, 0.6875, 0.6625])

In [13]:
clf.cost_complexity_pruning_path(X, High)

{'ccp_alphas': array([0.        , 0.01247253, 0.01595901, 0.01859445, 0.03425053,
        0.04454237, 0.06185323, 0.10922562]),
 'impurities': array([0.67960272, 0.69207525, 0.70803426, 0.72662872, 0.76087925,
        0.80542162, 0.86727485, 0.97650047])}